In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml


import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
from openff.toolkit.topology import Molecule
import torch


import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

/Users/madilyn/miniconda3/envs/esp/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
/Users/madilyn/miniconda3/envs/esp/lib/python3.9/site-packages/qcelemental/models/results.py:788: DeprecationWarning: ResultProtocols has been renamed to AtomicResultProtocols and will be removed as soon as v0.13.0
  warn(
LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [2]:
# Use espaloma to find the FF parameters for Benzene
smiles = "c1ccccc1"
benzene = mb.load(smiles, smiles=True)
#benzene.visualize().show()

molecule = Molecule.from_smiles(smiles) #molecule is benzene
molecule_graph = esp.Graph(molecule)

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph)

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]

In [50]:
print(pair_forces.getParticleParameters(0)) 
#some issues with the data that comes out of pair_forces.getParticleParameters:
    # WE have  list of "QUantities", which is nice, but we only want the values (but need to know units)
    # WE need informatio nabout which particle type each of these quanitities corresponds to
help(openmm_system.getForces)

[Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole)]
Help on method getForces in module openmm.openmm:

getForces() method of openmm.openmm.System instance
    Get the list of Forces in this System



In [7]:
#structure.bonds

In [3]:
# get a parmed structure from openmm 
import parmed as pmd
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()

structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)

In [4]:
# step 1: how to get bond parameters

bond_parameters = {}

for bond in structure.bonds:
    bond_parameters[(bond.atom1.name, bond.atom2.name)] = {"k":[bond.type.k] , "l0":[bond.type.req]}
    
# step 2: get angle parameters

angle_parameters = {}

for angle in structure.angles:
    angle_parameters[(angle.atom1.name, angle.atom2.name,angle.atom3.name)] = {"k":[angle.type.k], 
                                                                               "t0":[angle.type.theteq]}
    
# step 3: dihedral parameters

dihedral_parameters = {}

for dihedral in structure.dihedrals:
    dihedral_parameters[(dihedral.atom1.name, dihedral.atom2.name,dihedral.atom3.name,
                         dihedral.atom4.name)] = {"periodicity":[dihedral.type.per],
                                                  "k":[dihedral.type.phi_k],"phase":[dihedral.type.phase]}
    
# step 4: non-bonding parameters

nonbonded_parameters = {}

for nonbonded in structure.adjusts:
    nonbonded_parameters[(nonbonded.atom1.name,nonbonded.atom2.name)] = {"sigma":[nonbonded.type.rmin],
                                                    "epsilon":[nonbonded.type.epsilon],
                                                    "charge":[nonbonded.type.chgscale]}

In [ ]:
#bond_parameters
#angle_parameters
#dihedral_parameters
#nonbonded_parameters

In [ ]:
mbuild_to_foyer_xml(
    file_name="benzene_esp.xml",
    compound=benzene,
    bond_params=bond_parameters,
    angle_params=angle_parameters,
    dihedral_params=dihedral_parameters,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_parameters,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

In [ ]:
##creating espaloma xml without parmed

In [46]:
particle_types = []
particle_type_dict = dict()
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    pair_parms = pair_forces.getParticleParameters(index=i)
    charge = pair_parms[0]
    sigma = pair_parms[1]
    epsilon = pair_parms[2]
    if (charge, sigma, epsilon) not in particle_types: #this is adding the charge,sigma, and epsilon to the particle_types, but only once.
        particle_types.append((charge, sigma, epsilon))
        print((charge, sigma, epsilon) )
    particle_type_dict[i] = particle_types.index((charge, sigma, epsilon)) #adding only the index of the pair_parms, this gives particle types.
#print(particle_type_dict)

#particle_types gives the sigma, charge, and epsilon associated with each unique particle, 
#particle_type_dict only gives the index associated with each unique particle type

(Quantity(value=-0.13, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole))
(Quantity(value=0.13, unit=elementary charge), Quantity(value=0.25996424595335105, unit=nanometer), Quantity(value=0.06276, unit=kilojoule/mole))


In [44]:
bond_types = []
bond_dict = dict() #creating an empty dictionary to put our parameters in 
atom1_names = {}

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    atoms = {(bond.atom1.name,bond.atom2.name):[]} #gives you the atom type name and index
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    for bond in structure.bonds:
        atom1 = bond.atom1.name
        atom2 = bond.atom2.name
        if (atom1,atom2,k,l0) not in bond_types:
            bond_types.append((atom1,atom2,k,l0))
        bond_dict = bond_types
#print(bond_dict)

In [45]:
angle_types = []
angle_dict = dict() #creating an empty dictionary to put our parameters in 

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    atoms = {(angle.atom1.name,angle.atom2.name, angle.atom3.name):[]} #gives you the atom type name and index
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit
    for angle in structure.angles:
        atom1 = angle.atom1.name
        atom2 = angle.atom2.name
        atom3 = angle.atom3.name
        #if (atom1,atom2,atom3,k,t0) not in angle_types:
        #    angle_types.append((atom1,atom2,atom3,k,t0))
        angle_dict[(atom1,atom2,atom3)] = {"k":[k],"t0":[t0]}
#print(angle_dict)

In [55]:
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
    for nonbonded in structure.adjusts:
        atom1 = nonbonded.atom1.name
        atom2 = nonbonded.atom2.name
        nonbonded_dict[(atom1,atom2)] = {"charge":[charge],"sigma":[sigma],"epsilon":[epsilon]}
print(nonbonded_dict)

{('C1x', 'C4x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C2x', 'C5x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C3x', 'C6x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C3x', 'H1x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C5x', 'H1x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C4x', 'H2x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C6x', 'H2x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('H1x', 'H2x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C1x', 'H3x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C5x', 'H3x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('H2x', 'H3x'): {'charge': [0.13], 'sigma': [0.25996424595335105], 'epsilon': [0.06276]}, ('C2x', '

In [42]:
##using parmed to get atom types
#
#atom1_names = {}
#for bond in structure.bonds:
#    atom1_names = {"atom1":[bond.atom1.name,bond.atom1.idx],"atom2":[bond.atom2.name, bond.atom2.idx]} #gives you the atom type name and index
##    print(atom1_names)
#atoms = {}
#for bond in structure.bonds:
#    atoms = {(bond.atom1.name,bond.atom2.name):[bond.atom1.idx,bond.atom2.idx]} #gives you the atom type name and index
#    print(atoms)

In [47]:
dihedral_parameters

{('C1x', 'C2x', 'C3x', 'C4x'): {'periodicity': [6],
  'k': [0.02235557875720232],
  'phase': [0.0]},
 ('C1x', 'C2x', 'C3x', 'H3x'): {'periodicity': [6],
  'k': [0.023185446266553934],
  'phase': [180.0]},
 ('C1x', 'C6x', 'C5x', 'C4x'): {'periodicity': [6],
  'k': [0.02235557875720232],
  'phase': [0.0]},
 ('C1x', 'C6x', 'C5x', 'H5x'): {'periodicity': [6],
  'k': [0.023185446266553934],
  'phase': [180.0]},
 ('C2x', 'C1x', 'C6x', 'C5x'): {'periodicity': [6],
  'k': [0.02235557875720232],
  'phase': [0.0]},
 ('C2x', 'C1x', 'C6x', 'H6x'): {'periodicity': [6],
  'k': [0.023185446266553934],
  'phase': [180.0]},
 ('C2x', 'C3x', 'C4x', 'C5x'): {'periodicity': [6],
  'k': [0.02235557875720232],
  'phase': [0.0]},
 ('C2x', 'C3x', 'C4x', 'H4x'): {'periodicity': [6],
  'k': [0.023185446266553934],
  'phase': [180.0]},
 ('C3x', 'C2x', 'C1x', 'C6x'): {'periodicity': [6],
  'k': [0.02235557875720232],
  'phase': [0.0]},
 ('C3x', 'C2x', 'C1x', 'H1x'): {'periodicity': [6],
  'k': [0.02318544626655393